In [1]:
import pandas as pd
import numpy as np
infile="/Users/irffanalahi/Research/Research_update/SM/melcfdref/LargerLTME/BluOurPBL/RD/forfinalcollect/allfinals/BlOurPBLourCD8TIL_maxcompperc75percetile/getAUC/dashboard/AUC_and_Region/prepare_regionfile/shortname_reversedprioritization2/compartmentCD8TIL_response.txt_PreTxDCBvsNDB_AUC_othersRevePrio2.txt"

deltacol='maxCompartmentwisedelta'
qcol='q'
deltaoffset=0.03
qoffset=3
outfile=infile+"_binwith_offset_neglogq"+str(qoffset)+"_"+deltacol+str(deltaoffset)+".txt"
indf=pd.read_csv(infile,sep="\t")
indf['npqlog'] = -np.log10(indf[qcol])
adjustedqcolname='adjusted'+qcol
adjusteddeltacolname='adjusted'+deltacol
adjustedAUCcolname='adjustedAUC'
indf[adjustedqcolname]=-1
indf[adjusteddeltacolname]=-9999
indf[adjustedAUCcolname]=-1
indf.head()

,chrom,start,end,c,Region,q,diff,#cpg,p(MWU),p(2dks),...,CD8TIL-mDC,CD8TIL-mNeu,CD8TIL-cB,CD8TIL-MEL_TUMOR,maxCompartmentwisedelta,minCompartmentwiseDelta,npqlog,adjustedq,adjustedmaxCompartmentwisedelta,adjustedAUC
0,chr8,664479,665977,0.42,repeat,2.838900e-36,-0.621422,31,3.974600e-14,1.617800e-41,...,-0.673811,-0.680741,-0.541231,-0.611618,-0.505850,-0.687306,35.546850,-1,-9999,-1
1,chr4,77608166,77608745,0.50,repeat,2.297600e-16,-0.498407,16,4.229900e-14,1.511400e-20,...,-0.520482,-0.485712,-0.433899,-0.478316,-0.433899,-0.520691,15.638726,-1,-9999,-1
2,chr1,7932658,7933198,0.60,repeat,6.283900e-09,-0.210518,17,4.518600e-14,2.262900e-12,...,-0.265024,-0.248318,-0.214592,0.352976,0.352976,-0.277239,8.201771,-1,-9999,-1
3,chr10,124281041,124282218,0.64,repeat,1.170900e-08,-0.130548,22,5.040400e-14,4.647400e-12,...,-0.236801,-0.219059,-0.030468,0.468184,0.468184,-0.247289,7.931480,-1,-9999,-1
4,chr5,157344083,157344794,0.60,repeat,1.286800e-18,-0.647712,15,4.540800e-14,5.750400e-23,...,-0.723957,-0.745446,-0.581179,-0.630112,-0.482757,-0.747034,17.890489,-1,-9999,-1


In [2]:
def Average(lst):
    return sum(lst) / len(lst)

In [3]:
grouped=indf.groupby('Region')
for name, group in grouped:
    for gi, grow in group.iterrows():
        if grow['adjustedAUC']==-1:
            qhighest=grow['npqlog']+qoffset
            qlowest=grow['npqlog']-qoffset
            diffhighest=grow[deltacol]+deltaoffset
            difflowest=grow[deltacol]-deltaoffset
            
            similarows=group[(group['npqlog']>=qlowest) & (group['npqlog']<=qhighest) & (group[deltacol]>=difflowest) & (group[deltacol]<=diffhighest)]
            avgq_similarows=Average(similarows[qcol].tolist())
            avgdiff_similarows=Average(similarows[deltacol].tolist())
            avgAUC_similarows=Average(similarows['c'].tolist())
            
            corresindex=similarows.index
           
            
            indf.loc[corresindex,adjustedqcolname]=avgq_similarows
            indf.loc[corresindex,adjusteddeltacolname]=avgdiff_similarows
            indf.loc[corresindex,adjustedAUCcolname]=avgAUC_similarows
            

       

    

In [4]:
indf.rename(columns={qcol: "prevq", deltacol: "prev"+deltacol,"c":"prevc"},inplace=True,errors='raise')
indf.rename(columns={adjustedqcolname: qcol, adjusteddeltacolname:deltacol,adjustedAUCcolname:"c"},inplace=True,errors='raise')
indf.drop(['npqlog'],inplace=True,axis=1)

indf.to_csv(outfile,sep="\t",index=False)



